In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install necessary libraries (if not already installed)
# !pip install --upgrade albumentations timm

import os
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import timm
from sklearn.metrics import confusion_matrix
import numpy as np
from tqdm import tqdm
import multiprocessing

import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image

# Define data directories
train_dir = '/content/drive/MyDrive/train/'
val_dir = '/content/drive/MyDrive/val/val/'

# Robust data augmentation
train_transforms = A.Compose([
    A.Resize(224, 224),
    A.OneOf([
        A.RandomBrightnessContrast(),
        A.ColorJitter(),
        A.InvertImg(),
    ], p=0.8),
    A.OneOf([
        A.HorizontalFlip(),
        A.VerticalFlip(),
        A.RandomRotate90(),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=45),
        A.Affine(shear=(-30, 30)),
        A.Rotate(limit=45),
        A.CoarseDropout(max_holes=8, max_height=8, max_width=8, fill_value=0, p=0.5),
        A.Transpose(),
    ], p=0.8),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

val_transforms = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

# Custom dataset to use Albumentations
class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.dataset = ImageFolder(root=root_dir)
        self.transform = transform

    def __getitem__(self, idx):
        img_path, label = self.dataset.samples[idx]
        image = np.array(Image.open(img_path).convert('RGB'))
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed['image']
        return image, label

    def __len__(self):
        return len(self.dataset)

# Load datasets
train_dataset = CustomImageDataset(root_dir=train_dir, transform=train_transforms)
val_dataset = CustomImageDataset(root_dir=val_dir, transform=val_transforms)

# Verify class-to-label mapping
print("Class-to-Label Mapping for Training Dataset:")
print(train_dataset.dataset.class_to_idx)

# DataLoaders
num_workers = multiprocessing.cpu_count() - 1
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=num_workers, pin_memory=True)

# Define models with dropout
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Swin Transformer Model with Dropout and Weight Decay
model_swin = timm.create_model('swin_base_patch4_window7_224', pretrained=True, num_classes=2, drop_rate=0.2)
model_swin = model_swin.to(device)

# RegNetY Model with Dropout and Weight Decay
model_regnet = timm.create_model('regnety_032', pretrained=True, num_classes=2, drop_rate=0.2)
model_regnet = model_regnet.to(device)

# Define loss and optimizer with weight decay
criterion = nn.CrossEntropyLoss()
optimizer_swin = torch.optim.AdamW(model_swin.parameters(), lr=1e-4, weight_decay=1e-5)
optimizer_regnet = torch.optim.AdamW(model_regnet.parameters(), lr=1e-4, weight_decay=1e-5)

# Learning rate scheduler - Cosine Annealing
scheduler_swin = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_swin, T_max=50)
scheduler_regnet = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_regnet, T_max=50)

# Training parameters
num_epochs = 50

# For weighted ensemble
best_val_accuracy_swin = 0
best_val_accuracy_regnet = 0
best_val_accuracy_ensemble = 0

for epoch in range(num_epochs):
    model_swin.train()
    model_regnet.train()
    running_loss_swin = 0.0
    running_loss_regnet = 0.0

    # Training loop
    for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        # Zero the parameter gradients
        optimizer_swin.zero_grad()
        optimizer_regnet.zero_grad()

        # Forward pass
        outputs_swin = model_swin(images)
        outputs_regnet = model_regnet(images)

        # Compute loss
        loss_swin = criterion(outputs_swin, labels)
        loss_regnet = criterion(outputs_regnet, labels)
        total_loss = loss_swin + loss_regnet

        # Backward pass and optimization
        total_loss.backward()
        optimizer_swin.step()
        optimizer_regnet.step()

        running_loss_swin += loss_swin.item()
        running_loss_regnet += loss_regnet.item()

    # Calculate average training losses
    avg_loss_swin = running_loss_swin / len(train_loader)
    avg_loss_regnet = running_loss_regnet / len(train_loader)
    avg_total_loss = avg_loss_swin + avg_loss_regnet
    print(f'Epoch [{epoch+1}/{num_epochs}] Average Training Loss - Swin: {avg_loss_swin:.4f}, RegNetY: {avg_loss_regnet:.4f}')

    # Validation loop
    model_swin.eval()
    model_regnet.eval()
    val_loss_swin = 0.0
    val_loss_regnet = 0.0
    correct_swin = 0
    correct_regnet = 0
    correct_ensemble = 0
    total = 0
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            outputs_swin = model_swin(images)
            outputs_regnet = model_regnet(images)

            # Compute validation loss
            loss_swin = criterion(outputs_swin, labels)
            loss_regnet = criterion(outputs_regnet, labels)
            val_loss_swin += loss_swin.item()
            val_loss_regnet += loss_regnet.item()

            # Individual model predictions
            _, predicted_swin = torch.max(outputs_swin.data, 1)
            _, predicted_regnet = torch.max(outputs_regnet.data, 1)

            # Ensemble by averaging the outputs
            outputs = (outputs_swin + outputs_regnet) / 2
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct_swin += (predicted_swin == labels).sum().item()
            correct_regnet += (predicted_regnet == labels).sum().item()
            correct_ensemble += (predicted == labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    # Calculate average validation losses and accuracy
    avg_val_loss_swin = val_loss_swin / len(val_loader)
    avg_val_loss_regnet = val_loss_regnet / len(val_loader)
    avg_val_loss = avg_val_loss_swin + avg_val_loss_regnet
    val_accuracy_swin = 100 * correct_swin / total
    val_accuracy_regnet = 100 * correct_regnet / total
    val_accuracy_ensemble = 100 * correct_ensemble / total
    print(f'Validation Loss - Swin: {avg_val_loss_swin:.4f}, RegNetY: {avg_val_loss_regnet:.4f}, Total: {avg_val_loss:.4f}')
    print(f'Validation Accuracy - Swin: {val_accuracy_swin:.2f}%, RegNetY: {val_accuracy_regnet:.2f}%, Ensemble: {val_accuracy_ensemble:.2f}%')

    # Update learning rate schedulers
    scheduler_swin.step()
    scheduler_regnet.step()

    # Save the model based on best ensemble validation accuracy
    if val_accuracy_ensemble > best_val_accuracy_ensemble:
        best_val_accuracy_ensemble = val_accuracy_ensemble
        # Save the best model
        torch.save({
            'epoch': epoch + 1,
            'model_swin_state_dict': model_swin.state_dict(),
            'model_regnet_state_dict': model_regnet.state_dict(),
            'optimizer_swin_state_dict': optimizer_swin.state_dict(),
            'optimizer_regnet_state_dict': optimizer_regnet.state_dict(),
            'val_accuracy_ensemble': best_val_accuracy_ensemble,
            'best_val_accuracy_swin': val_accuracy_swin,
            'best_val_accuracy_regnet': val_accuracy_regnet,
        }, 'best_model.pth')
        print(f'Best model saved at epoch {epoch+1} with ensemble validation accuracy {best_val_accuracy_ensemble:.2f}%')

        # Update best validation accuracies
        best_val_accuracy_swin = val_accuracy_swin
        best_val_accuracy_regnet = val_accuracy_regnet
    else:
        print('No improvement in ensemble validation accuracy.')

print('Training completed.')

# Load the best model
checkpoint = torch.load('best_model.pth')
model_swin.load_state_dict(checkpoint['model_swin_state_dict'])
model_regnet.load_state_dict(checkpoint['model_regnet_state_dict'])
best_val_accuracy_swin = checkpoint['best_val_accuracy_swin']
best_val_accuracy_regnet = checkpoint['best_val_accuracy_regnet']

# Evaluate on the validation set
model_swin.eval()
model_regnet.eval()
all_labels = []
all_preds = []
correct_swin = 0
correct_regnet = 0
correct_ensemble = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        outputs_swin = model_swin(images)
        outputs_regnet = model_regnet(images)

        # Individual model predictions
        _, predicted_swin = torch.max(outputs_swin.data, 1)
        _, predicted_regnet = torch.max(outputs_regnet.data, 1)

        # Ensemble by weighted averaging based on best validation accuracies
        weight_swin = best_val_accuracy_swin / (best_val_accuracy_swin + best_val_accuracy_regnet)
        weight_regnet = best_val_accuracy_regnet / (best_val_accuracy_swin + best_val_accuracy_regnet)
        outputs = (weight_swin * outputs_swin + weight_regnet * outputs_regnet)

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct_swin += (predicted_swin == labels).sum().item()
        correct_regnet += (predicted_regnet == labels).sum().item()
        correct_ensemble += (predicted == labels).sum().item()

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Compute final accuracies
final_accuracy_swin = 100 * correct_swin / total
final_accuracy_regnet = 100 * correct_regnet / total
final_accuracy_ensemble = 100 * correct_ensemble / total
print(f'Final Validation Accuracy - Swin: {final_accuracy_swin:.2f}%, RegNetY: {final_accuracy_regnet:.2f}%, Ensemble: {final_accuracy_ensemble:.2f}%')

# Compute confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print('Confusion Matrix:')
print(cm)


Class-to-Label Mapping for Training Dataset:
{'nevus': 0, 'others': 1}


Epoch 1/50: 100%|██████████| 475/475 [03:51<00:00,  2.06it/s]

Epoch [1/50] Average Training Loss - Swin: 0.4209, RegNetY: 0.4320


Validation Loss - Swin: 0.3404, RegNetY: 0.3754, Total: 0.7158
Validation Accuracy - Swin: 85.01%, RegNetY: 83.59%, Ensemble: 84.69%
Best model saved at epoch 1 with ensemble validation accuracy 84.69%


Epoch 2/50: 100%|██████████| 475/475 [02:38<00:00,  3.00it/s]

...

Epoch 50/50: 100%|██████████| 475/475 [02:35<00:00,  3.05it/s]

Epoch [50/50] Average Training Loss - Swin: 0.0076, RegNetY: 0.0186


Validation Loss - Swin: 0.4769, RegNetY: 0.4088, Total: 0.8856
Validation Accuracy - Swin: 92.31%, RegNetY: 91.44%, Ensemble: 93.20%
No improvement in ensemble validation accuracy.
Training completed.


<ipython-input-4-e0a10f372afb>:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('best_model.pth')


Final Validation Accuracy - Swin: 92.49%, RegNetY: 91.49%, Ensemble: 93.20%
Confusion Matrix:
[[1822  109]
 [ 149 1716]]


In [ ]:
# Import necessary libraries
import os
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import timm
from sklearn.metrics import confusion_matrix, cohen_kappa_score
import numpy as np
from tqdm import tqdm
import multiprocessing

import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image

# Define data directories
train_dir = '/content/drive/MyDrive/train/'
val_dir = '/content/drive/MyDrive/val/val/'

# Data augmentation for validation (usually minimal)
val_transforms = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

# Custom dataset to use Albumentations
class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.dataset = ImageFolder(root=root_dir)
        self.transform = transform

    def __getitem__(self, idx):
        img_path, label = self.dataset.samples[idx]
        image = np.array(Image.open(img_path).convert('RGB'))
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed['image']
        return image, label

    def __len__(self):
        return len(self.dataset)

# Load datasets
val_dataset = CustomImageDataset(root_dir=val_dir, transform=val_transforms)

# Verify class-to-label mapping
print("Class-to-Label Mapping for Validation Dataset:")
print(val_dataset.dataset.class_to_idx)

# DataLoader
num_workers = multiprocessing.cpu_count() - 1
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=num_workers, pin_memory=True)

# Define models with dropout
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Swin Transformer Model
model_swin = timm.create_model('swin_base_patch4_window7_224', pretrained=False, num_classes=2, drop_rate=0.2)
model_swin = model_swin.to(device)

# RegNetY Model
model_regnet = timm.create_model('regnety_032', pretrained=False, num_classes=2, drop_rate=0.2)
model_regnet = model_regnet.to(device)

# Load the best model checkpoint
checkpoint_path = '/content/drive/MyDrive/Model/best_model.pth'  # Update this path if necessary
checkpoint = torch.load(checkpoint_path, map_location=device)
model_swin.load_state_dict(checkpoint['model_swin_state_dict'])
model_regnet.load_state_dict(checkpoint['model_regnet_state_dict'])

# Move models to evaluation mode
model_swin.eval()
model_regnet.eval()

# Initialize variables to track metrics
all_labels = []
all_preds_ensemble = []
all_preds_swin = []
all_preds_regnet = []
correct_swin = 0
correct_regnet = 0
correct_ensemble = 0
total = 0

# Retrieve best validation accuracies for weighted ensemble
best_val_accuracy_swin = checkpoint.get('best_val_accuracy_swin', 0)
best_val_accuracy_regnet = checkpoint.get('best_val_accuracy_regnet', 0)

# Ensure that the sum of weights is not zero
weight_sum = best_val_accuracy_swin + best_val_accuracy_regnet
if weight_sum == 0:
    weight_swin = weight_regnet = 0.5
else:
    weight_swin = best_val_accuracy_swin / weight_sum
    weight_regnet = best_val_accuracy_regnet / weight_sum

print(f'Using weights for ensemble - Swin: {weight_swin:.4f}, RegNetY: {weight_regnet:.4f}')

# Evaluation loop
with torch.no_grad():
    for images, labels in tqdm(val_loader, desc='Evaluating'):
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        # Forward pass
        outputs_swin = model_swin(images)
        outputs_regnet = model_regnet(images)

        # Individual model predictions
        _, predicted_swin = torch.max(outputs_swin.data, 1)
        _, predicted_regnet = torch.max(outputs_regnet.data, 1)

        # Ensemble by weighted averaging based on best validation accuracies
        outputs_ensemble = (weight_swin * outputs_swin + weight_regnet * outputs_regnet)
        _, predicted_ensemble = torch.max(outputs_ensemble.data, 1)

        # Update total and correct counts
        total += labels.size(0)
        correct_swin += (predicted_swin == labels).sum().item()
        correct_regnet += (predicted_regnet == labels).sum().item()
        correct_ensemble += (predicted_ensemble == labels).sum().item()

        # Collect all labels and predictions for metrics
        all_labels.extend(labels.cpu().numpy())
        all_preds_swin.extend(predicted_swin.cpu().numpy())
        all_preds_regnet.extend(predicted_regnet.cpu().numpy())
        all_preds_ensemble.extend(predicted_ensemble.cpu().numpy())

# Compute final accuracies
final_accuracy_swin = 100 * correct_swin / total
final_accuracy_regnet = 100 * correct_regnet / total
final_accuracy_ensemble = 100 * correct_ensemble / total

print(f'\nFinal Validation Accuracy:')
print(f' - Swin Transformer: {final_accuracy_swin:.2f}%')
print(f' - RegNetY: {final_accuracy_regnet:.2f}%')
print(f' - Ensemble: {final_accuracy_ensemble:.2f}%')

# Compute confusion matrix for ensemble
cm = confusion_matrix(all_labels, all_preds_ensemble)
print('\nConfusion Matrix for Ensemble:')
print(cm)

# Compute Cohen's Kappa score for ensemble
kappa = cohen_kappa_score(all_labels, all_preds_ensemble)
print(f'\nCohen\'s Kappa Score for Ensemble: {kappa:.4f}')


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.22 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Class-to-Label Mapping for Validation Dataset:
{'nevus': 0, 'others': 1}


<ipython-input-2-800efa4e6411>:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


Using weights for ensemble - Swin: 0.5027, RegNetY: 0.4973


Evaluating: 100%|██████████| 119/119 [03:41<00:00,  1.86s/it]


Final Validation Accuracy:
 - Swin Transformer: 92.49%
 - RegNetY: 91.49%
 - Ensemble: 93.20%

Confusion Matrix for Ensemble:
[[1822  109]
 [ 149 1716]]

Cohen's Kappa Score for Ensemble: 0.8640


# Hybrid Model

In [ ]:
# Install necessary libraries (if not already installed)
# !pip install --upgrade albumentations timm

import os
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import timm
from sklearn.metrics import confusion_matrix
import numpy as np
from tqdm import tqdm
import multiprocessing

import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image

# Define data directories
train_dir = '/content/drive/MyDrive/train/'
val_dir = '/content/drive/MyDrive/val/val/'

# Robust data augmentation
train_transforms = A.Compose([
    A.Resize(224, 224),
    A.OneOf([
        A.RandomBrightnessContrast(),
        A.ColorJitter(),
        A.InvertImg(),
    ], p=0.8),
    A.OneOf([
        A.HorizontalFlip(),
        A.VerticalFlip(),
        A.RandomRotate90(),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=45),
        A.Affine(shear=(-30, 30)),
        A.Rotate(limit=45),
        A.CoarseDropout(max_holes=8, max_height=8, max_width=8, fill_value=0, p=0.5),
        A.Transpose(),
    ], p=0.8),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

val_transforms = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

# Custom dataset to use Albumentations
class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.dataset = ImageFolder(root=root_dir)
        self.transform = transform

    def __getitem__(self, idx):
        img_path, label = self.dataset.samples[idx]
        image = np.array(Image.open(img_path).convert('RGB'))
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed['image']
        return image, label

    def __len__(self):
        return len(self.dataset)

# Load datasets
train_dataset = CustomImageDataset(root_dir=train_dir, transform=train_transforms)
val_dataset = CustomImageDataset(root_dir=val_dir, transform=val_transforms)

# Verify class-to-label mapping
print("Class-to-Label Mapping for Training Dataset:")
print(train_dataset.dataset.class_to_idx)

# DataLoaders
num_workers = multiprocessing.cpu_count() - 1
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=num_workers, pin_memory=True)

# Define models with dropout
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Swin Transformer Model with Dropout
model_swin = timm.create_model('swin_base_patch4_window7_224', pretrained=True, num_classes=2, drop_rate=0.2)
model_swin = model_swin.to(device)

# EfficientNet Model with Dropout
model_efficientnet = timm.create_model('tf_efficientnet_b7_ns', pretrained=True, num_classes=2, drop_rate=0.2)
model_efficientnet = model_efficientnet.to(device)

# Define loss and optimizer with weight decay
criterion = nn.CrossEntropyLoss()
optimizer_swin = torch.optim.AdamW(model_swin.parameters(), lr=1e-4, weight_decay=1e-5)
optimizer_efficientnet = torch.optim.AdamW(model_efficientnet.parameters(), lr=1e-4, weight_decay=1e-5)

# Modified LR schedulers - Using CosineAnnealingWarmRestarts
scheduler_swin = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_swin, T_0=10, T_mult=2)
scheduler_efficientnet = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_efficientnet, T_0=10, T_mult=2)

# Training parameters
num_epochs = 50

# For weighted ensemble
best_val_accuracy_swin = 0
best_val_accuracy_efficientnet = 0
best_val_accuracy_ensemble = 0

for epoch in range(num_epochs):
    model_swin.train()
    model_efficientnet.train()
    running_loss_swin = 0.0
    running_loss_efficientnet = 0.0

    # Training loop
    for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        # Zero the parameter gradients
        optimizer_swin.zero_grad()
        optimizer_efficientnet.zero_grad()

        # Forward pass
        outputs_swin = model_swin(images)
        outputs_efficientnet = model_efficientnet(images)

        # Compute loss
        loss_swin = criterion(outputs_swin, labels)
        loss_efficientnet = criterion(outputs_efficientnet, labels)
        total_loss = loss_swin + loss_efficientnet

        # Backward pass and optimization
        total_loss.backward()
        optimizer_swin.step()
        optimizer_efficientnet.step()

        running_loss_swin += loss_swin.item()
        running_loss_efficientnet += loss_efficientnet.item()

    # Calculate average training losses
    avg_loss_swin = running_loss_swin / len(train_loader)
    avg_loss_efficientnet = running_loss_efficientnet / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}] Average Training Loss - Swin: {avg_loss_swin:.4f}, EfficientNet: {avg_loss_efficientnet:.4f}')

    # Validation loop
    model_swin.eval()
    model_efficientnet.eval()
    val_loss_swin = 0.0
    val_loss_efficientnet = 0.0
    correct_swin = 0
    correct_efficientnet = 0
    correct_ensemble = 0
    total = 0
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            outputs_swin = model_swin(images)
            outputs_efficientnet = model_efficientnet(images)

            # Compute validation loss
            loss_swin = criterion(outputs_swin, labels)
            loss_efficientnet = criterion(outputs_efficientnet, labels)
            val_loss_swin += loss_swin.item()
            val_loss_efficientnet += loss_efficientnet.item()

            # Individual model predictions
            _, predicted_swin = torch.max(outputs_swin.data, 1)
            _, predicted_efficientnet = torch.max(outputs_efficientnet.data, 1)

            # Ensemble by averaging the outputs
            outputs = (outputs_swin + outputs_efficientnet) / 2
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct_swin += (predicted_swin == labels).sum().item()
            correct_efficientnet += (predicted_efficientnet == labels).sum().item()
            correct_ensemble += (predicted == labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    # Calculate average validation losses and accuracy
    avg_val_loss_swin = val_loss_swin / len(val_loader)
    avg_val_loss_efficientnet = val_loss_efficientnet / len(val_loader)
    val_accuracy_swin = 100 * correct_swin / total
    val_accuracy_efficientnet = 100 * correct_efficientnet / total
    val_accuracy_ensemble = 100 * correct_ensemble / total
    print(f'Validation Loss - Swin: {avg_val_loss_swin:.4f}, EfficientNet: {avg_val_loss_efficientnet:.4f}')
    print(f'Validation Accuracy - Swin: {val_accuracy_swin:.2f}%, EfficientNet: {val_accuracy_efficientnet:.2f}%, Ensemble: {val_accuracy_ensemble:.2f}%')

    # Update learning rate schedulers with warm restarts
    scheduler_swin.step(epoch + 1)           # step with the epoch count
    scheduler_efficientnet.step(epoch + 1)   # step with the epoch count

    # Save the model based on best ensemble validation accuracy
    if val_accuracy_ensemble > best_val_accuracy_ensemble:
        best_val_accuracy_ensemble = val_accuracy_ensemble
        # Save the best model
        torch.save({
            'epoch': epoch + 1,
            'model_swin_state_dict': model_swin.state_dict(),
            'model_efficientnet_state_dict': model_efficientnet.state_dict(),
            'optimizer_swin_state_dict': optimizer_swin.state_dict(),
            'optimizer_efficientnet_state_dict': optimizer_efficientnet.state_dict(),
            'val_accuracy_ensemble': best_val_accuracy_ensemble,
            'best_val_accuracy_swin': val_accuracy_swin,
            'best_val_accuracy_efficientnet': val_accuracy_efficientnet,
        }, 'best_model.pth')
        print(f'Best model saved at epoch {epoch+1} with ensemble validation accuracy {best_val_accuracy_ensemble:.2f}%')

        # Update best validation accuracies
        best_val_accuracy_swin = val_accuracy_swin
        best_val_accuracy_efficientnet = val_accuracy_efficientnet
    else:
        print('No improvement in ensemble validation accuracy.')

print('Training completed.')

# Load the best model
checkpoint = torch.load('best_model.pth')
model_swin.load_state_dict(checkpoint['model_swin_state_dict'])
model_efficientnet.load_state_dict(checkpoint['model_efficientnet_state_dict'])
best_val_accuracy_swin = checkpoint['best_val_accuracy_swin']
best_val_accuracy_efficientnet = checkpoint['best_val_accuracy_efficientnet']

# Evaluate on the validation set with weighted ensemble
model_swin.eval()
model_efficientnet.eval()
all_labels = []
all_preds = []
correct_swin = 0
correct_efficientnet = 0
correct_ensemble = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        outputs_swin = model_swin(images)
        outputs_efficientnet = model_efficientnet(images)

        # Individual model predictions
        _, predicted_swin = torch.max(outputs_swin.data, 1)
        _, predicted_efficientnet = torch.max(outputs_efficientnet.data, 1)

        # Ensemble by weighted averaging based on best validation accuracies
        weight_swin = best_val_accuracy_swin / (best_val_accuracy_swin + best_val_accuracy_efficientnet)
        weight_efficientnet = best_val_accuracy_efficientnet / (best_val_accuracy_swin + best_val_accuracy_efficientnet)
        outputs = (weight_swin * outputs_swin + weight_efficientnet * outputs_efficientnet)

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct_swin += (predicted_swin == labels).sum().item()
        correct_efficientnet += (predicted_efficientnet == labels).sum().item()
        correct_ensemble += (predicted == labels).sum().item()

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Compute final accuracies
final_accuracy_swin = 100 * correct_swin / total
final_accuracy_efficientnet = 100 * correct_efficientnet / total
final_accuracy_ensemble = 100 * correct_ensemble / total
print(f'Final Validation Accuracy - Swin: {final_accuracy_swin:.2f}%, EfficientNet: {final_accuracy_efficientnet:.2f}%, Ensemble: {final_accuracy_ensemble:.2f}%')

# Compute confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print('Confusion Matrix:')
print(cm)


Class-to-Label Mapping for Training Dataset:
{'nevus': 0, 'others': 1}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b7_ns to current tf_efficientnet_b7.ns_jft_in1k.
  model = create_fn(


model.safetensors:   0%|          | 0.00/267M [00:00<?, ?B/s]

...

Epoch 50/50: 100%|██████████| 475/475 [03:38<00:00,  2.17it/s]

Epoch [50/50] Average Training Loss - Swin: 0.0329, EfficientNet: 0.0238


Validation Loss - Swin: 0.3781, EfficientNet: 0.5065
Validation Accuracy - Swin: 91.57%, EfficientNet: 90.73%, Ensemble: 92.12%
No improvement in ensemble validation accuracy.
Training completed.


<ipython-input-3-40997c491869>:226: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('best_model.pth')


Final Validation Accuracy - Swin: 92.39%, EfficientNet: 91.02%, Ensemble: 93.02%
Confusion Matrix:
[[1806  125]
 [ 140 1725]]


# Three Class Problem

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install necessary libraries (if not already installed)
# !pip install --upgrade albumentations timm

import os
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, WeightedRandomSampler
import timm
from sklearn.metrics import confusion_matrix, cohen_kappa_score
import numpy as np
from tqdm import tqdm
import multiprocessing

import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image

# ADDED for copying file to Drive
import shutil

#######################################
# Data Directories and Class Mapping
#######################################
# Data directories:
train_dir = '/content/drive/MyDrive/Three_Class/train_3/train_3/'
val_dir = '/content/drive/MyDrive/Three_Class/val_3/val_3/'

# Desired mapping: mel=0, bcc=1, scc=2
# Alphabetical order: bcc, mel, scc → original indexing: bcc=0, mel=1, scc=2
# We must remap:
# mel (original 1) → 0
# bcc (original 0) → 1
# scc (original 2) → 2
original_to_desired = {0: 1, 1: 0, 2: 2}

#######################################
# Data Transforms
#######################################
train_transforms = A.Compose([
    A.Resize(224, 224),
    A.OneOf([
        A.RandomBrightnessContrast(),
        A.ColorJitter(),
        A.InvertImg(),
    ], p=0.8),
    A.OneOf([
        A.HorizontalFlip(),
        A.VerticalFlip(),
        A.RandomRotate90(),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=45),
        A.Affine(shear=(-30, 30)),
        A.Rotate(limit=45),
        A.CoarseDropout(max_holes=8, max_height=8, max_width=8, fill_value=0, p=0.5),
        A.Transpose(),
    ], p=0.8),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

val_transforms = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

#######################################
# Custom Dataset
#######################################
class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None, label_map=None):
        self.dataset = ImageFolder(root=root_dir)
        self.transform = transform
        self.label_map = label_map  # Dictionary for remapping labels

    def __getitem__(self, idx):
        img_path, label = self.dataset.samples[idx]
        image = np.array(Image.open(img_path).convert('RGB'))
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed['image']

        # Remap label using original_to_desired mapping if provided
        if self.label_map is not None:
            label = self.label_map[label]

        return image, label

    def __len__(self):
        return len(self.dataset)

#######################################
# Load Datasets
#######################################
train_dataset = CustomImageDataset(root_dir=train_dir, transform=train_transforms, label_map=original_to_desired)
val_dataset = CustomImageDataset(root_dir=val_dir, transform=val_transforms, label_map=original_to_desired)

print("Class-to-Label Mapping for Training Dataset (original):")
print(train_dataset.dataset.class_to_idx)
print("Our desired mapping is mel=0, bcc=1, scc=2.")

#######################################
# Handle Class Imbalance with WeightedRandomSampler
#######################################
# We now compute class frequencies from the train_dataset
all_train_labels = [train_dataset[i][1] for i in range(len(train_dataset))]
class_counts = np.bincount(all_train_labels)
# class_counts[i] = number of samples for class i
# Example: mel=0, bcc=1, scc=2

class_weights = 1.0 / class_counts
# Create a weight for each sample based on its class
sample_weights = [class_weights[label] for label in all_train_labels]

sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

#######################################
# DataLoaders
#######################################
num_workers = multiprocessing.cpu_count() - 1
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=num_workers, pin_memory=True)

#######################################
# Model Setup
#######################################
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Modify num_classes=3 for mel, bcc, scc classification
model_swin = timm.create_model('swin_base_patch4_window7_224', pretrained=True, num_classes=3, drop_rate=0.2)
model_swin = model_swin.to(device)

model_regnet = timm.create_model('regnety_032', pretrained=True, num_classes=3, drop_rate=0.2)
model_regnet = model_regnet.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer_swin = torch.optim.AdamW(model_swin.parameters(), lr=1e-4, weight_decay=1e-5)
optimizer_regnet = torch.optim.AdamW(model_regnet.parameters(), lr=1e-4, weight_decay=1e-5)

# Learning rate schedulers
scheduler_swin = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_swin, T_max=50)
scheduler_regnet = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_regnet, T_max=50)

num_epochs = 100

best_val_accuracy_swin = 0
best_val_accuracy_regnet = 0
best_val_accuracy_ensemble = 0

#######################################
# Drive folder to copy model
#######################################
# Change this path to your desired Google Drive folder:
drive_folder = '/content/drive/MyDrive/Three_Class/trained_models'
os.makedirs(drive_folder, exist_ok=True)  # Create folder if it doesn't exist

#######################################
# Training and Validation Loop
#######################################
for epoch in range(num_epochs):
    model_swin.train()
    model_regnet.train()
    running_loss_swin = 0.0
    running_loss_regnet = 0.0

    # Training loop
    for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        # Zero gradients
        optimizer_swin.zero_grad()
        optimizer_regnet.zero_grad()

        # Forward pass
        outputs_swin = model_swin(images)
        outputs_regnet = model_regnet(images)

        # Compute loss
        loss_swin = criterion(outputs_swin, labels)
        loss_regnet = criterion(outputs_regnet, labels)
        total_loss = loss_swin + loss_regnet

        # Backward + Optimize
        total_loss.backward()
        optimizer_swin.step()
        optimizer_regnet.step()

        running_loss_swin += loss_swin.item()
        running_loss_regnet += loss_regnet.item()

    # Average training losses
    avg_loss_swin = running_loss_swin / len(train_loader)
    avg_loss_regnet = running_loss_regnet / len(train_loader)
    avg_total_loss = avg_loss_swin + avg_loss_regnet
    print(f'Epoch [{epoch+1}/{num_epochs}] Training Loss - Swin: {avg_loss_swin:.4f}, RegNetY: {avg_loss_regnet:.4f}')

    # Validation
    model_swin.eval()
    model_regnet.eval()
    val_loss_swin = 0.0
    val_loss_regnet = 0.0
    correct_swin = 0
    correct_regnet = 0
    correct_ensemble = 0
    total = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            outputs_swin = model_swin(images)
            outputs_regnet = model_regnet(images)

            loss_swin = criterion(outputs_swin, labels)
            loss_regnet = criterion(outputs_regnet, labels)
            val_loss_swin += loss_swin.item()
            val_loss_regnet += loss_regnet.item()

            # Predictions
            _, predicted_swin = torch.max(outputs_swin, 1)
            _, predicted_regnet = torch.max(outputs_regnet, 1)

            # Simple averaging for ensemble
            combined_outputs = (outputs_swin + outputs_regnet) / 2
            _, predicted_ensemble = torch.max(combined_outputs, 1)

            total += labels.size(0)
            correct_swin += (predicted_swin == labels).sum().item()
            correct_regnet += (predicted_regnet == labels).sum().item()
            correct_ensemble += (predicted_ensemble == labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted_ensemble.cpu().numpy())

    # Validation metrics
    avg_val_loss_swin = val_loss_swin / len(val_loader)
    avg_val_loss_regnet = val_loss_regnet / len(val_loader)
    avg_val_loss = avg_val_loss_swin + avg_val_loss_regnet
    val_accuracy_swin = 100 * correct_swin / total
    val_accuracy_regnet = 100 * correct_regnet / total
    val_accuracy_ensemble = 100 * correct_ensemble / total

    # Calculate validation Kappa (using the ensemble predictions)
    val_kappa_ensemble = cohen_kappa_score(all_labels, all_preds)

    print(f'Validation Loss - Swin: {avg_val_loss_swin:.4f}, '
          f'RegNetY: {avg_val_loss_regnet:.4f}, Total: {avg_val_loss:.4f}')
    print(f'Validation Accuracy - Swin: {val_accuracy_swin:.2f}%, '
          f'RegNetY: {val_accuracy_regnet:.2f}%, '
          f'Ensemble: {val_accuracy_ensemble:.2f}%, '
          f'Kappa (Ensemble): {val_kappa_ensemble:.4f}')

    # Update schedulers
    scheduler_swin.step()
    scheduler_regnet.step()

    # Save best model based on the best ensemble accuracy
    if val_accuracy_ensemble > best_val_accuracy_ensemble:
        best_val_accuracy_ensemble = val_accuracy_ensemble
        torch.save({
            'epoch': epoch + 1,
            'model_swin_state_dict': model_swin.state_dict(),
            'model_regnet_state_dict': model_regnet.state_dict(),
            'optimizer_swin_state_dict': optimizer_swin.state_dict(),
            'optimizer_regnet_state_dict': optimizer_regnet.state_dict(),
            'val_accuracy_ensemble': best_val_accuracy_ensemble,
            'best_val_accuracy_swin': val_accuracy_swin,
            'best_val_accuracy_regnet': val_accuracy_regnet,
        }, 'best_model.pth')

        # Copy the saved model to Drive
        drive_model_path = os.path.join(drive_folder, 'best_model.pth')
        shutil.copy('best_model.pth', drive_model_path)

        print(f'Best model saved at epoch {epoch+1} with ensemble validation accuracy {best_val_accuracy_ensemble:.2f}%')
        print(f'Model also copied to: {drive_model_path}')

        best_val_accuracy_swin = val_accuracy_swin
        best_val_accuracy_regnet = val_accuracy_regnet
    else:
        print('No improvement in ensemble validation accuracy.')

print('Training completed.')

#######################################
# Final Evaluation with Weighted Ensemble
#######################################
checkpoint = torch.load('best_model.pth', map_location=device)
model_swin.load_state_dict(checkpoint['model_swin_state_dict'])
model_regnet.load_state_dict(checkpoint['model_regnet_state_dict'])
best_val_accuracy_swin = checkpoint['best_val_accuracy_swin']
best_val_accuracy_regnet = checkpoint['best_val_accuracy_regnet']

model_swin.eval()
model_regnet.eval()

all_labels = []
all_preds = []
correct_swin = 0
correct_regnet = 0
correct_ensemble = 0
total = 0

# Weighted ensemble based on best validation accuracies
weight_sum = best_val_accuracy_swin + best_val_accuracy_regnet
if weight_sum == 0:
    weight_swin = 0.5
    weight_regnet = 0.5
else:
    weight_swin = best_val_accuracy_swin / weight_sum
    weight_regnet = best_val_accuracy_regnet / weight_sum

print(f'Using weighted ensemble: Swin Weight={weight_swin:.2f}, RegNetY Weight={weight_regnet:.2f}')

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        outputs_swin = model_swin(images)
        outputs_regnet = model_regnet(images)

        # Individual predictions
        _, predicted_swin = torch.max(outputs_swin, 1)
        _, predicted_regnet = torch.max(outputs_regnet, 1)

        # Weighted ensemble
        weighted_outputs = weight_swin * outputs_swin + weight_regnet * outputs_regnet
        _, predicted_ensemble = torch.max(weighted_outputs, 1)

        total += labels.size(0)
        correct_swin += (predicted_swin == labels).sum().item()
        correct_regnet += (predicted_regnet == labels).sum().item()
        correct_ensemble += (predicted_ensemble == labels).sum().item()

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted_ensemble.cpu().numpy())

final_accuracy_swin = 100 * correct_swin / total
final_accuracy_regnet = 100 * correct_regnet / total
final_accuracy_ensemble = 100 * correct_ensemble / total
print(f'Final Validation Accuracy - Swin: {final_accuracy_swin:.2f}%, '
      f'RegNetY: {final_accuracy_regnet:.2f}%, '
      f'Ensemble: {final_accuracy_ensemble:.2f}%')

cm = confusion_matrix(all_labels, all_preds)
final_kappa_ensemble = cohen_kappa_score(all_labels, all_preds)
print('Confusion Matrix:')
print(cm)
print(f'Final Kappa (Ensemble): {final_kappa_ensemble:.4f}')


Class-to-Label Mapping for Training Dataset (original):
{'bcc': 0, 'mel': 1, 'scc': 2}
Our desired mapping is mel=0, bcc=1, scc=2.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/78.1M [00:00<?, ?B/s]

Epoch 1/100: 100%|██████████| 159/159 [00:56<00:00,  2.80it/s]

...

Epoch 100/100: 100%|██████████| 159/159 [00:53<00:00,  2.99it/s]

Epoch [100/100] Training Loss - Swin: 0.0309, RegNetY: 0.0354


Validation Loss - Swin: 0.2674, RegNetY: 0.2491, Total: 0.5165
Validation Accuracy - Swin: 93.62%, RegNetY: 94.57%, Ensemble: 95.20%, Kappa (Ensemble): 0.9125
No improvement in ensemble validation accuracy.
Training completed.


<ipython-input-3-7976b97a4d9a>:295: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('best_model.pth', map_location=device)


Using weighted ensemble: Swin Weight=0.50, RegNetY Weight=0.50
Final Validation Accuracy - Swin: 94.88%, RegNetY: 94.41%, Ensemble: 96.93%
Confusion Matrix:
[[659  16   3]
 [  8 488   2]
 [  3   7  84]]
Final Kappa (Ensemble): 0.9447
